In [88]:
import pickle
import pdb
import codecs
import re
import sys
import math
import numpy as np

In [89]:
with open('../Bosondata.pkl', 'rb') as inp:
	word2id = pickle.load(inp)
	id2word = pickle.load(inp)
	tag2id = pickle.load(inp)
	id2tag = pickle.load(inp)
	x_train = pickle.load(inp)
	y_train = pickle.load(inp)
	x_test = pickle.load(inp)
	y_test = pickle.load(inp)
	x_valid = pickle.load(inp)
	y_valid = pickle.load(inp)
print("train len:",len(x_train))
print("valid len:",len(x_valid))
print("test len:",len(x_test))
print("word2id len", len(word2id))
print('Creating the data generator ...')

train len: 10721
valid len: 2681
test len: 3351
word2id len 3435
Creating the data generator ...


In [90]:
y_train = np.expand_dims(y_train , 2)
y_valid = np.expand_dims(y_valid , 2)
y_test = np.expand_dims(y_test , 2)

In [91]:
x_train.shape , y_train.shape

((10721, 60), (10721, 60, 1))

In [92]:
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM
from keras_contrib.layers import CRF
import pickle

EMBED_DIM = 200 # 词向量的维度
BiRNN_UNITS = 200

In [93]:
model = Sequential()
model.add(Embedding(len(word2id), EMBED_DIM, mask_zero=True))  # Random embedding
model.add(Bidirectional(LSTM(BiRNN_UNITS // 2, return_sequences=True)))
crf = CRF(len(tag2id), sparse_target=True)
model.add(crf)
model.summary()
model.compile('adam', loss=crf.loss_function, metrics=[crf.accuracy])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 200)         687000    
_________________________________________________________________
bidirectional_3 (Bidirection (None, None, 200)         240800    
_________________________________________________________________
crf_3 (CRF)                  (None, None, 20)          4460      
Total params: 932,260
Trainable params: 932,260
Non-trainable params: 0
_________________________________________________________________


D:\Anaconda\envs\tfenv\lib\site-packages\keras_contrib\layers\crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
D:\Anaconda\envs\tfenv\lib\site-packages\keras_contrib\layers\crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


In [94]:
EPOCHS = 10
# train model
model.fit(x_train , y_train , batch_size=16 , epochs=EPOCHS , validation_data=[x_valid, y_valid])
model.save('./crf_Boson.h5')

Train on 10721 samples, validate on 2681 samples
Epoch 1/10
10721/10721 [==============================] - 150s 14ms/step - loss: 17.4071 - crf_viterbi_accuracy: 0.7368 - val_loss: 17.1719 - val_crf_viterbi_accuracy: 0.8266
Epoch 2/10
10721/10721 [==============================] - 109s 10ms/step - loss: 16.7614 - crf_viterbi_accuracy: 0.8566 - val_loss: 17.0099 - val_crf_viterbi_accuracy: 0.8589
Epoch 3/10
10721/10721 [==============================] - 114s 11ms/step - loss: 16.6213 - crf_viterbi_accuracy: 0.8926 - val_loss: 16.9561 - val_crf_viterbi_accuracy: 0.8636
Epoch 4/10
10721/10721 [==============================] - 127s 12ms/step - loss: 16.5539 - crf_viterbi_accuracy: 0.9102 - val_loss: 16.9291 - val_crf_viterbi_accuracy: 0.8750
Epoch 5/10
10721/10721 [==============================] - 106s 10ms/step - loss: 16.5147 - crf_viterbi_accuracy: 0.9276 - val_loss: 16.9172 - val_crf_viterbi_accuracy: 0.8759
Epoch 6/10
10721/10721 [==============================] - 98s 9ms/step - los

In [96]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 16.6285643610304
Test accuracy: 0.8842257091942847


In [97]:
maxlen=60 # 最大句长
predict_text = '''中华人民共和国国务院总理周恩来在外交部长陈毅的陪同下，
连续访问了埃塞俄比亚等非洲10国以及阿尔巴尼亚'''

In [98]:
from keras.preprocessing.sequence import pad_sequences
x = [word2id.get(w[0].lower(), 1) for w in predict_text]
length = len(x)
x = pad_sequences([x], maxlen, padding='post')
model.load_weights('./crf_Boson.h5')
raw = model.predict(x)[0][:length]
result = [np.argmax(row) for row in raw]
result_tags = [id2tag[i] for i in result]

In [99]:
product_name, time, company_name, location, person_name, org_name = '', '', '', '', '', ''

for s, t in zip(predict_text, result_tags):
    if t in ('B_product_name', 'M_product_name', 'E_product_name'):
        product_name += ' ' + s if (t == 'B_product_name') else s
    if t in ('B_time', 'M_time', 'E_time'):
        time += ' ' + s if (t == 'B_time') else s
    if t in ('B_company_name', 'M_company_name', 'E_company_name'):
        company_name += ' ' + s if (t == 'B_company_name') else s
    if t in ('B_location', 'M_location', 'E_location'):
        location += ' ' + s if (t == 'B_location') else s
    if t in ('B_person_name', 'M_person_name', 'E_person_name'):
        person_name += ' ' + s if (t == 'B_person_name') else s
    if t in ('B_org_name', 'M_org_name', 'E_org_name'):
        org_name += ' ' + s if (t == 'B_org_name') else s

print(' product_name:' + product_name, '\n',
      'time:' + time, '\n',
      'company_name:' + company_name, '\n',
      'location:' + location, '\n',
      'person_name:' + person_name, '\n',
      'org_name:' + org_name)

 product_name: 
 time: 
 company_name: 阿尔巴尼亚 
 location: 埃塞俄比亚 
 person_name: 周恩来 陈毅 
 org_name: 中华人民共和国国务院
